Tutorial on how to visualize and also analyze the waveforms:
https://github.com/multichannelsystems/McsPyDataTools/blob/master/McsPyDataNotebooks/McsPy-Tutorial_DataAnalysis.ipynb

In [3]:
!pip list

Package            Version
------------------ -----------
asttokens          3.0.0
colorama           0.4.6
comm               0.2.3
debugpy            1.8.17
decorator          5.2.1
exceptiongroup     1.3.0
executing          2.2.1
importlib_metadata 8.7.0
ipykernel          7.0.0
ipython            8.37.0
jedi               0.19.2
jupyter_client     8.6.3
jupyter_core       5.8.1
matplotlib-inline  0.1.7
nest_asyncio       1.6.0
packaging          25.0
parso              0.8.5
pickleshare        0.7.5
pip                25.2
platformdirs       4.5.0
prompt_toolkit     3.0.52
psutil             7.1.0
pure_eval          0.2.3
Pygments           2.19.2
python-dateutil    2.9.0.post0
pywin32            311
pyzmq              27.1.0
setuptools         80.9.0
six                1.17.0
stack_data         0.6.3
tornado            6.5.2
traitlets          5.14.3
typing_extensions  4.15.0
wcwidth            0.2.14
wheel              0.45.1
zipp               3.23.0


In [5]:
!pip install -r requirements.txt


  Using cached spikeinterface-0.103.0-py3-none-any.whl.metadata (11 kB)
  Using cached quantities-0.16.2-py3-none-any.whl.metadata (8.4 kB)
  Using cached McsPyDataTools-0.4.3.tar.gz (2.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached probeinterface-0.3.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached asciitree-0.3.3.tar.gz (4.0 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached fasteners-0.20-py3-none-any.whl.metadata (4.8 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached flexcache-0.3-py3-none-any.whl.metadata (7.0 kB)
  Using cached flexparser-0.4-py3-none-any.whl.metadata (18 kB)
Using cached spikeinterface-0.103.0-py3-none-any.whl (1.1 MB)
   ---------------------------------------- 0.0/828.6 kB ? eta -:--:--
   ---------------------------------------- 828.6/828.6 kB 35.4 MB/s  0:00:00
   ------------------

  DEPRECATION: Building 'McsPyDataTools' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'McsPyDataTools'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'asciitree' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'asciitree'. Discussion can be found at https://github.com/pypa/pip/issues/6334


Imports

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# MCS PyData tools
import McsPy
import McsPy.McsData
from McsPy import ureg, Q_

# VISUALIZATION TOOLS
import matplotlib.pyplot as plt
%matplotlib inline

# SUPRESS WARNINGS
import warnings
warnings.filterwarnings('ignore')

# autoreload modules
%load_ext autoreload
%autoreload 2

Functions 

In [4]:
def plot_analog_stream_channel(analog_stream, channel_idx, from_in_s=0, to_in_s=None, show=True):
    """
    Plots data from a single AnalogStream channel
    
    :param analog_stream: A AnalogStream object
    :param channel_idx: A scalar channel index (0 <= channel_idx < # channels in the AnalogStream)
    :param from_in_s: The start timestamp of the plot (0 <= from_in_s < to_in_s). Default: 0
    :param to_in_s: The end timestamp of the plot (from_in_s < to_in_s <= duration). Default: None (= recording duration)
    :param show: If True (default), the plot is directly created. For further plotting, use show=False
    """
    # extract basic information
    ids = [c.channel_id for c in analog_stream.channel_infos.values()]
    channel_id = ids[channel_idx]
    channel_info = analog_stream.channel_infos[channel_id]
    sampling_frequency = channel_info.sampling_frequency.magnitude
   
    # get start and end index
    from_idx = max(0, int(from_in_s * sampling_frequency))
    if to_in_s is None:
        to_idx = analog_stream.channel_data.shape[1]
    else:
        to_idx = min(analog_stream.channel_data.shape[1], int(to_in_s * sampling_frequency))
        
    # get the timestamps for each sample
    time = analog_stream.get_channel_sample_timestamps(channel_id, from_idx, to_idx)

    # scale time to seconds:
    scale_factor_for_second = Q_(1,time[1]).to(ureg.s).magnitude
    time_in_sec = time[0] * scale_factor_for_second
    
    # get the signal
    signal = analog_stream.get_channel_in_range(channel_id, from_idx, to_idx)

    # scale signal to µV:
    scale_factor_for_uV = Q_(1,signal[1]).to(ureg.uV).magnitude
    signal_in_uV = signal[0] * scale_factor_for_uV

    # construct the plot
    _ = plt.figure(figsize=(20,6))
    _ = plt.plot(time_in_sec, signal_in_uV)
    _ = plt.xlabel('Time (%s)' % ureg.s)
    _ = plt.ylabel('Voltage (%s)' % ureg.uV)
    _ = plt.title('Channel %s' % channel_info.info['Label'])
    if show:
        plt.show()

Code to visualize and analyze HDF files

In [ ]:
FILE_PATH = " " # Insert with filename
file = file = McsPy.McsData.RawData(FILE_PATH)
electrode_stream = file.recordings[0].analog_streams[0];

In [ ]:
# Check the analog stream of the electrode in channel 0
plot_analog_stream_channel(electrode_stream, 0, from_in_s=0, to_in_s=500)

In [ ]:
# Check the bandwidth of the recording to see if
# there have been any filters applied o
channel_id = 171
info = electrode_stream.channel_infos[channel_id].info
print("Bandwidth: %s - %s Hz" % (info['HighPassFilterCutOffFrequency'], info['LowPassFilterCutOffFrequency']))

signal = electrode_stream.get_channel_in_range(channel_id, 0, electrode_stream.channel_data.shape[1])[0]

In [ ]:
# Determine suitable threshold for spike detection
noise_std = np.std(signal)
noise_mad = np.median(np.absolute(signal)) / 0.6745
print('Noise Estimate by Standard Deviation: {0:g} V'.format(noise_std))
print('Noise Estimate by MAD Estimator     : {0:g} V'.format(noise_mad))

In [ ]:
spike_threshold = -5 * noise_mad # roughly -30 µV


In [ ]:
# Plot the data with the spike threshold
plot_analog_stream_channel(electrode_stream, 9, from_in_s=155, to_in_s=200, show=False)
_ = plt.plot([155, 200], [spike_threshold*1e6, spike_threshold*1e6]) # converts the threshold to µV for plotting
plt.show()